## Time series generation using VAEs.
This is a minimal example of unsupervised time series generation using VAEs.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sklearn
import copy
import sklearn.model_selection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tsgm

#### 1. Choose architecture of encoder and decoder.
Here, you can either use one of the architectures presented in `tsgm.models.architectures`, or define custom discriminator and generator architectures as `tf` models.

In [ ]:
architecture = tsgm.models.zoo["vae_conv5"](24, 5, 10)
encoder, decoder = architecture.encoder, architecture.decoder

#### 2. Load data:
We are working with a toy dataset, and use `tsgm` utility called `tsgm.utils.gen_sine_dataset` to generate the data. Next, we featurewise scale the dataset so that each feature is in $[0, 1]$, using `tsgm.utils.TSFeatureWiseScaler`.

In [ ]:
data = tsgm.utils.gen_sine_dataset(10000, 24, 5)
scaler = tsgm.utils.TSFeatureWiseScaler()        
scaled_data = scaler.fit_transform(data)

#### 3. Define model and train it.
We define conditional GAN model (`tsgm.models.cvae.BetaVAE`), compile it, and train using `.fit` model. Additionally, we use `tsgm.models.monitors.GANMonitor` to track training process.

In [ ]:
vae = tsgm.models.cvae.BetaVAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

vae.fit(scaled_data, epochs=100, batch_size=64)

#### 4. Check reconstruction of the data.
We reconstruct data using `vae.predict(scaled_data)`. For validating that VAE works, we check that original and reconstructed datasets are visually similar using `tsgm.utils.visualize_original_and_reconst_ts`.

In [ ]:
x_decoded = vae.predict(scaled_data)

In [ ]:
tsgm.utils.visualize_original_and_reconst_ts(scaled_data, x_decoded, num=10)